<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generation-of-regular-polyhedrons" data-toc-modified-id="Generation-of-regular-polyhedrons-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generation of regular polyhedrons</a></span></li></ul></div>

# Generation of regular polyhedrons

The aim of this notebook is to generate a large list or regular polyhedra.

The first half of the polyhedra must have their vertices on a sphere of radius 1.,
the second half must have their vertices on a sphere of radius 1.5 .

The geometric center of each polyhedron must be on a cubic lattice.


LMGC90's preprocessor classic parts (model, material and containers)

In [ ]:
from pylmgc90 import pre

dim = 3

bodies = pre.avatars()
maters = pre.materials()

mod = pre.model(name='rigid', physics='MECAx', element='Rxx3D', dimension=dim)

mater = pre.material(name='PLExx', materialType='RIGID', density=100.)
maters+=mater

Some parameters of the script:

* mininum number of vertices
* maximum number of vertices
* the two radii to use

Which will allow to compute the number of polyhedra to generate as well as the size of the box:

In [ ]:
import math

min_vert = 4
max_vert = 200

rmin = 1.
rmax = 1.5

nb_poly = max_vert - min_vert + 1 # relies on the face that max_vert > min_vert
nb_box_ele = int( math.ceil( nb_poly**(1./3.) ) )

Generation of square lattice of square shape:

In [ ]:
coors = pre.cubicLattice3D( nb_box_ele, nb_box_ele, nb_box_ele, 2.*max(rmin,rmax) )
coors.shape = [coors.size//3,3]

First half of particles generation:

In [ ]:
for i in range(nb_poly//2):
    new_body = pre.rigidPolyhedron(mod, mater, nb_vertices=min_vert+i, radius=rmin)
    new_body.translate( dx=coors[i,0], dy=coors[i,1], dz=coors[i,2] )
    bodies += new_body

Second half of particles generation, but instead of translating the particle after generating it, let's just place the center of the polyhedron at generation:

In [ ]:
for i in range(nb_poly//2,nb_poly):
    new_body = pre.rigidPolyhedron(mod, mater, nb_vertices=min_vert+i, radius=rmax, center=coors[i,:])
    bodies += new_body

Visualisation:

In [ ]:
pre.visuAvatars(bodies)

LMGC90 reading and Paraview file generation:

In [ ]:
import os

if not os.path.isdir('DATBOX'):
    os.mkdir('DATBOX')

pre.writeBodies(bodies, chemin='DATBOX/')
pre.writeDrvDof(bodies, chemin='DATBOX/')
pre.writeDofIni(bodies, chemin='DATBOX/')
pre.writeBulkBehav(maters, chemin='DATBOX/', dim=dim)


In [ ]:
from pylmgc90 import chipy

chipy.checkDirectories()
chipy.Initialize()

chipy.SetDimension(3)

chipy.ReadBehaviours()
chipy.ReadBodies()
chipy.LoadBehaviours()
chipy.ReadIniDof()
chipy.ReadDrivenDof()
chipy.LoadTactors()

chipy.OpenDisplayFiles()
chipy.WriteDisplayFiles()
chipy.CloseDisplayFiles()

chipy.Finalize()